In [ ]:
import mysql.connector
from tkinter import *
from tkinter import ttk, messagebox
from datetime import datetime

DB_CONFIG = {
    "host": "localhost",
    "user": "root",
    "password": "Eshwar1@",  # <-- change this
    "database": "car_rental_db",
}

class Database:
    def __init__(self):
        self.conn = mysql.connector.connect(**DB_CONFIG)
        self.cur = self.conn.cursor()

    # ---------- Vehicles ----------
    def add_vehicle(self, make, model, year, rate):
        self.cur.execute(
            "INSERT INTO vehicles(make, model, year, rental_rate, is_available) VALUES(%s,%s,%s,%s,TRUE)",
            (make, model, year, rate)
        )
        self.conn.commit()

    def get_vehicles(self):
        self.cur.execute("SELECT vehicle_id, make, model, year, rental_rate, is_available FROM vehicles ORDER BY vehicle_id DESC")
        return self.cur.fetchall()

    def delete_vehicle(self, vehicle_id):
        self.cur.execute("DELETE FROM vehicles WHERE vehicle_id=%s", (vehicle_id,))
        self.conn.commit()

    def update_vehicle(self, vehicle_id, make, model, year, rate, is_available):
        self.cur.execute(
            "UPDATE vehicles SET make=%s, model=%s, year=%s, rental_rate=%s, is_available=%s WHERE vehicle_id=%s",
            (make, model, year, rate, is_available, vehicle_id)
        )
        self.conn.commit()

    # ---------- Customers ----------
    def add_customer(self, name, email, phone):
        self.cur.execute(
            "INSERT INTO customers(name, email, phone) VALUES(%s,%s,%s)",
            (name, email, phone)
        )
        self.conn.commit()

    def get_customers(self):
        self.cur.execute("SELECT customer_id, name, email, phone FROM customers ORDER BY customer_id DESC")
        return self.cur.fetchall()

    def delete_customer(self, customer_id):
        self.cur.execute("DELETE FROM customers WHERE customer_id=%s", (customer_id,))
        self.conn.commit()

    # ---------- Bookings ----------
    def is_vehicle_available(self, vehicle_id, start_date, end_date):
        # Check availability via overlap detection
        self.cur.execute(
            """
            SELECT COUNT(*) FROM bookings
            WHERE vehicle_id=%s AND NOT (end_date < %s OR start_date > %s)
            """,
            (vehicle_id, start_date, end_date)
        )
        (count,) = self.cur.fetchone()
        # Also ensure vehicle marked available
        self.cur.execute("SELECT is_available FROM vehicles WHERE vehicle_id=%s", (vehicle_id,))
        row = self.cur.fetchone()
        flag = bool(row[0]) if row else False
        return count == 0 and flag

    def get_vehicle_rate(self, vehicle_id):
        self.cur.execute("SELECT rental_rate FROM vehicles WHERE vehicle_id=%s", (vehicle_id,))
        row = self.cur.fetchone()
        return float(row[0]) if row else 0.0

    def create_booking(self, customer_id, vehicle_id, start_date, end_date, total_cost):
        self.cur.execute(
            "INSERT INTO bookings(customer_id, vehicle_id, start_date, end_date, total_cost) VALUES(%s,%s,%s,%s,%s)",
            (customer_id, vehicle_id, start_date, end_date, total_cost)
        )
        # Optionally mark vehicle unavailable for period by setting flag false
        self.cur.execute("UPDATE vehicles SET is_available=FALSE WHERE vehicle_id=%s", (vehicle_id,))
        self.conn.commit()

    def get_bookings(self):
        self.cur.execute(
            """
            SELECT b.booking_id, c.name, v.make, v.model, b.start_date, b.end_date, b.total_cost
            FROM bookings b
            JOIN customers c ON b.customer_id=c.customer_id
            JOIN vehicles v ON b.vehicle_id=v.vehicle_id
            ORDER BY b.booking_id DESC
            """
        )
        return self.cur.fetchall()

    def close(self):
        self.cur.close()
        self.conn.close()

class CarRentalApp:
    def __init__(self, root):
        self.db = Database()
        self.root = root
        self.root.title("Car Rental System")
        self.root.geometry("1000x650")

        nb = ttk.Notebook(root)
        nb.pack(fill=BOTH, expand=True)

        self.vehicles_tab = Frame(nb)
        self.customers_tab = Frame(nb)
        self.bookings_tab = Frame(nb)

        nb.add(self.vehicles_tab, text="Vehicles")
        nb.add(self.customers_tab, text="Customers")
        nb.add(self.bookings_tab, text="Bookings")

        self.build_vehicles_tab()
        self.build_customers_tab()
        self.build_bookings_tab()

        self.root.protocol("WM_DELETE_WINDOW", self.on_close)

    # ---------------- Vehicles UI ----------------
    def build_vehicles_tab(self):
        form = LabelFrame(self.vehicles_tab, text="Add / Update Vehicle", padx=10, pady=10)
        form.pack(fill=X, padx=10, pady=10)

        Label(form, text="Make").grid(row=0, column=0, sticky=W)
        Label(form, text="Model").grid(row=0, column=2, sticky=W)
        Label(form, text="Year").grid(row=1, column=0, sticky=W)
        Label(form, text="Rate").grid(row=1, column=2, sticky=W)
        Label(form, text="Available").grid(row=0, column=4, sticky=W)

        self.v_make = Entry(form, width=20)
        self.v_model = Entry(form, width=20)
        self.v_year = Entry(form, width=10)
        self.v_rate = Entry(form, width=10)
        self.v_available = ttk.Combobox(form, values=["Yes", "No"], width=7)
        self.v_available.current(0)

        self.v_make.grid(row=0, column=1, padx=5, pady=5)
        self.v_model.grid(row=0, column=3, padx=5, pady=5)
        self.v_year.grid(row=1, column=1, padx=5, pady=5)
        self.v_rate.grid(row=1, column=3, padx=5, pady=5)
        self.v_available.grid(row=0, column=5, padx=5, pady=5)

        btn_frame = Frame(form)
        btn_frame.grid(row=2, column=0, columnspan=6, sticky=E, pady=5)
        Button(btn_frame, text="Add", command=self.add_vehicle, bg="#2e7d32", fg="white").pack(side=LEFT, padx=5)
        Button(btn_frame, text="Update", command=self.update_vehicle).pack(side=LEFT, padx=5)
        Button(btn_frame, text="Delete", command=self.delete_vehicle, bg="#c62828", fg="white").pack(side=LEFT, padx=5)
        Button(btn_frame, text="Clear", command=self.clear_vehicle_form).pack(side=LEFT, padx=5)

        # table
        self.v_tree = ttk.Treeview(self.vehicles_tab, columns=("id","make","model","year","rate","avail"), show="headings")
        for col, txt, w in (
            ("id","ID",60), ("make","Make",150), ("model","Model",150), ("year","Year",80), ("rate","Rate",100), ("avail","Available",100)
        ):
            self.v_tree.heading(col, text=txt)
            self.v_tree.column(col, width=w, anchor=CENTER)
        self.v_tree.pack(fill=BOTH, expand=True, padx=10, pady=10)
        self.v_tree.bind("<<TreeviewSelect>>", self.on_vehicle_select)

        self.refresh_vehicles()

    def add_vehicle(self):
        try:
            make = self.v_make.get().strip()
            model = self.v_model.get().strip()
            year = int(self.v_year.get().strip())
            rate = float(self.v_rate.get().strip())
            if not make or not model:
                raise ValueError("Make/Model required")
            self.db.add_vehicle(make, model, year, rate)
            messagebox.showinfo("Success", "Vehicle added")
            self.refresh_vehicles(); self.clear_vehicle_form()
        except Exception as e:
            messagebox.showerror("Error", str(e))

    def update_vehicle(self):
        sel = self.v_tree.selection()
        if not sel:
            messagebox.showwarning("Select", "Select a vehicle to update")
            return
        vid = int(self.v_tree.item(sel[0], 'values')[0])
        try:
            make = self.v_make.get().strip()
            model = self.v_model.get().strip()
            year = int(self.v_year.get().strip())
            rate = float(self.v_rate.get().strip())
            is_avail = self.v_available.get() == "Yes"
            self.db.update_vehicle(vid, make, model, year, rate, is_avail)
            messagebox.showinfo("Updated", "Vehicle updated")
            self.refresh_vehicles()
        except Exception as e:
            messagebox.showerror("Error", str(e))

    def delete_vehicle(self):
        sel = self.v_tree.selection()
        if not sel:
            messagebox.showwarning("Select", "Select a vehicle to delete")
            return
        vid = int(self.v_tree.item(sel[0], 'values')[0])
        if messagebox.askyesno("Confirm", "Delete selected vehicle?"):
            try:
                self.db.delete_vehicle(vid)
                self.refresh_vehicles(); self.clear_vehicle_form()
            except Exception as e:
                messagebox.showerror("Error", str(e))

    def refresh_vehicles(self):
        for i in self.v_tree.get_children():
            self.v_tree.delete(i)
        for row in self.db.get_vehicles():
            self.v_tree.insert('', END, values=(row[0], row[1], row[2], row[3], f"{row[4]:.2f}", "Yes" if row[5] else "No"))

    def clear_vehicle_form(self):
        self.v_make.delete(0, END); self.v_model.delete(0, END)
        self.v_year.delete(0, END); self.v_rate.delete(0, END)
        self.v_available.set("Yes")

    def on_vehicle_select(self, _):
        sel = self.v_tree.selection()
        if not sel:
            return
        vals = self.v_tree.item(sel[0], 'values')
        self.v_make.delete(0, END); self.v_make.insert(0, vals[1])
        self.v_model.delete(0, END); self.v_model.insert(0, vals[2])
        self.v_year.delete(0, END); self.v_year.insert(0, vals[3])
        self.v_rate.delete(0, END); self.v_rate.insert(0, vals[4])
        self.v_available.set(vals[5])

    # ---------------- Customers UI ----------------
    def build_customers_tab(self):
        form = LabelFrame(self.customers_tab, text="Add Customer", padx=10, pady=10)
        form.pack(fill=X, padx=10, pady=10)

        Label(form, text="Name").grid(row=0, column=0, sticky=W)
        Label(form, text="Email").grid(row=0, column=2, sticky=W)
        Label(form, text="Phone").grid(row=0, column=4, sticky=W)

        self.c_name = Entry(form, width=25)
        self.c_email = Entry(form, width=25)
        self.c_phone = Entry(form, width=15)

        self.c_name.grid(row=0, column=1, padx=5, pady=5)
        self.c_email.grid(row=0, column=3, padx=5, pady=5)
        self.c_phone.grid(row=0, column=5, padx=5, pady=5)

        Button(form, text="Add", command=self.add_customer, bg="#2e7d32", fg="white").grid(row=0, column=6, padx=10)

        self.c_tree = ttk.Treeview(self.customers_tab, columns=("id","name","email","phone"), show="headings")
        for col, txt, w in (("id","ID",60),("name","Name",200),("email","Email",250),("phone","Phone",150)):
            self.c_tree.heading(col, text=txt)
            self.c_tree.column(col, width=w, anchor=CENTER)
        self.c_tree.pack(fill=BOTH, expand=True, padx=10, pady=10)

        self.refresh_customers()

    def add_customer(self):
        try:
            name = self.c_name.get().strip()
            email = self.c_email.get().strip()
            phone = self.c_phone.get().strip()
            if not name:
                raise ValueError("Name required")
            self.db.add_customer(name, email, phone)
            messagebox.showinfo("Success", "Customer added")
            self.c_name.delete(0, END); self.c_email.delete(0, END); self.c_phone.delete(0, END)
            self.refresh_customers()
        except Exception as e:
            messagebox.showerror("Error", str(e))

    def refresh_customers(self):
        for i in self.c_tree.get_children():
            self.c_tree.delete(i)
        for row in self.db.get_customers():
            self.c_tree.insert('', END, values=row)

    # ---------------- Bookings UI ----------------
    def build_bookings_tab(self):
        form = LabelFrame(self.bookings_tab, text="Create Booking", padx=10, pady=10)
        form.pack(fill=X, padx=10, pady=10)

        Label(form, text="Customer").grid(row=0, column=0, sticky=W)
        Label(form, text="Vehicle (ID)").grid(row=0, column=2, sticky=W)
        Label(form, text="Start (YYYY-MM-DD)").grid(row=1, column=0, sticky=W)
        Label(form, text="End (YYYY-MM-DD)").grid(row=1, column=2, sticky=W)

        # Customer dropdown by id-name
        customers = self.db.get_customers()
        self.b_customer = ttk.Combobox(form, values=[f"{c[0]} - {c[1]}" for c in customers], width=28)
        self.b_customer.grid(row=0, column=1, padx=5, pady=5)

        # Vehicle dropdown by id-make-model only those marked available
        vehicles = [v for v in self.db.get_vehicles() if v[5]]
        self.b_vehicle = ttk.Combobox(form, values=[f"{v[0]} - {v[1]} {v[2]}" for v in vehicles], width=28)
        self.b_vehicle.grid(row=0, column=3, padx=5, pady=5)

        self.b_start = Entry(form, width=20)
        self.b_end = Entry(form, width=20)
        self.b_start.grid(row=1, column=1, padx=5, pady=5)
        self.b_end.grid(row=1, column=3, padx=5, pady=5)

        Button(form, text="Create", command=self.create_booking, bg="#1565c0", fg="white").grid(row=2, column=3, sticky=E, padx=5, pady=10)

        self.b_tree = ttk.Treeview(self.bookings_tab, columns=("id","customer","make","model","start","end","cost"), show="headings")
        for col, txt, w in (("id","ID",60),("customer","Customer",200),("make","Make",120),("model","Model",120),("start","Start",110),("end","End",110),("cost","Cost",100)):
            self.b_tree.heading(col, text=txt)
            self.b_tree.column(col, width=w, anchor=CENTER)
        self.b_tree.pack(fill=BOTH, expand=True, padx=10, pady=10)

        self.refresh_bookings()

    def create_booking(self):
        try:
            if not self.b_customer.get() or not self.b_vehicle.get():
                raise ValueError("Select customer and vehicle")
            customer_id = int(self.b_customer.get().split(" - ")[0])
            vehicle_id = int(self.b_vehicle.get().split(" - ")[0])

            start = self.b_start.get().strip()
            end = self.b_end.get().strip()
            # Validate dates
            sdt = datetime.strptime(start, "%Y-%m-%d").date()
            edt = datetime.strptime(end, "%Y-%m-%d").date()
            if edt < sdt:
                raise ValueError("End date must be after start date")

            if not self.db.is_vehicle_available(vehicle_id, sdt, edt):
                raise ValueError("Vehicle not available for selected dates")

            days = (edt - sdt).days + 1
            rate = self.db.get_vehicle_rate(vehicle_id)
            total = round(days * rate, 2)

            self.db.create_booking(customer_id, vehicle_id, sdt, edt, total)
            messagebox.showinfo("Booked", f"Booking created. Total cost: {total}")
            self.refresh_bookings()
            self.refresh_vehicles()
            # Clear inputs
            self.b_start.delete(0, END); self.b_end.delete(0, END)
        except Exception as e:
            messagebox.showerror("Error", str(e))

    def refresh_bookings(self):
        for i in self.b_tree.get_children():
            self.b_tree.delete(i)
        for row in self.db.get_bookings():
            self.b_tree.insert('', END, values=(row[0], row[1], row[2], row[3], row[4], row[5], f"{row[6]:.2f}"))

    def on_close(self):
        self.db.close()
        self.root.destroy()

if __name__ == "__main__":
    root = Tk()
    app = CarRentalApp(root)
    root.mainloop()